# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_data="../output_data/cities.csv"
vacation_py=pd.read_csv(vacation_data)
vacation_py.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Buala,-8.1450,159.5921,77.68,89,100,4.99,SB,1612435814
1,Yellowknife,62.4560,-114.3525,-23.01,76,90,10.36,CA,1612435742
2,Kondinskoye,59.6481,67.4100,9.34,95,100,5.70,RU,1612435814
3,Lázaro Cárdenas,17.9583,-102.2000,72.27,85,53,3.83,MX,1612435814
4,Kapaa,22.0752,-159.3190,64.00,77,90,5.75,US,1612435814


In [3]:
vacation_py.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,5.590000e+02
mean,19.805845,17.149888,52.456780,71.110912,53.518784,8.230537,1.612436e+09
std,32.892679,91.090735,32.734254,21.566648,39.439159,5.982336,5.988774e+01
min,-54.800000,-179.166700,-50.800000,9.000000,0.000000,0.560000,1.612435e+09
25%,-7.256350,-62.535150,30.425000,61.000000,11.000000,3.725000,1.612436e+09
50%,23.463200,21.511300,64.400000,77.000000,63.000000,6.910000,1.612436e+09
75%,46.961050,100.080200,77.960000,86.500000,91.500000,10.970000,1.612436e+09
max,78.218600,179.316700,111.720000,100.000000,100.000000,35.300000,1.612436e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [50]:
locations = vacation_py[["Lat", "Lng"]].astype(float)
weight = vacation_py["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False,
                                 max_intensity=100,point_radius = 5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [52]:
plt.savefig("../Images/heatmap.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
max_weather=vacation_py.loc[(vacation_py["Max Temp"] < 80) & (vacation_py["Max Temp"] > 70)]
wind_weather=max_weather.loc[max_weather["Wind Speed"] < 10]
ideal_weather=wind_weather.loc[wind_weather["Cloudiness"] == 0]
ideal_weather.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Atuona,-9.8000,-139.0333,77.58,73,0,9.64,PF,1612435817
43,Abu Dhabi,24.4667,54.3667,78.80,50,0,6.91,AE,1612435819
67,Resistencia,-27.4606,-58.9839,72.00,94,0,4.61,AR,1612435822
70,Kiryat Gat,31.6100,34.7642,71.60,46,0,6.91,IL,1612435823
148,Zihuatanejo,17.6333,-101.5500,73.04,78,0,4.12,MX,1612435832
234,Pilar,-9.5972,-35.9567,75.20,94,0,3.44,BR,1612435719
331,Puerto Escondido,15.8500,-97.0667,74.28,74,0,4.41,MX,1612435855
337,São João da Barra,-21.6403,-41.0511,77.00,78,0,6.91,BR,1612435856
340,Sur,22.5667,59.5289,75.00,59,0,9.17,OM,1612435856
349,Tiarei,-17.5333,-149.3333,75.20,88,0,2.30,PF,1612435857


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\Spare\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Atuona,-9.8000,-139.0333,77.58,73,0,9.64,PF,1612435817,
43,Abu Dhabi,24.4667,54.3667,78.80,50,0,6.91,AE,1612435819,
67,Resistencia,-27.4606,-58.9839,72.00,94,0,4.61,AR,1612435822,
70,Kiryat Gat,31.6100,34.7642,71.60,46,0,6.91,IL,1612435823,
148,Zihuatanejo,17.6333,-101.5500,73.04,78,0,4.12,MX,1612435832,
234,Pilar,-9.5972,-35.9567,75.20,94,0,3.44,BR,1612435719,
331,Puerto Escondido,15.8500,-97.0667,74.28,74,0,4.41,MX,1612435855,
337,São João da Barra,-21.6403,-41.0511,77.00,78,0,6.91,BR,1612435856,
340,Sur,22.5667,59.5289,75.00,59,0,9.17,OM,1612435856,
349,Tiarei,-17.5333,-149.3333,75.20,88,0,2.30,PF,1612435857,


In [37]:
params = {
    "radius" : 5000,
    "type": "lodging",
    "key": g_key }
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    #get lat and lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    city = row['City']


    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 23: Atuona.
Closest hotel is Villa Enata.
------------
Retrieving Results for Index 43: Abu Dhabi.
Closest hotel is Beach Rotana.
------------
Retrieving Results for Index 67: Resistencia.
Closest hotel is Niyat Urban Hotel.
------------
Retrieving Results for Index 70: Kiryat Gat.
Closest hotel is OlusHome.
------------
Retrieving Results for Index 148: Zihuatanejo.
Closest hotel is La Casa Que Canta.
------------
Retrieving Results for Index 234: Pilar.
Closest hotel is Pousada e churrascaria Manguaba.
------------
Retrieving Results for Index 331: Puerto Escondido.
Closest hotel is Aldea del Bazar Hotel and Spa.
------------
Retrieving Results for Index 337: São João da Barra.
Closest hotel is Pousada Porto De Canoas.
------------
Retrieving Results for Index 340: Sur.
Closest hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 349: Tiarei.
Closest hotel is Le Rocher de Tahiti.
------------
Retrieving Results for Index 353: Tura.
Closest 

In [46]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Atuona,-9.8000,-139.0333,77.58,73,0,9.64,PF,1612435817,Villa Enata
43,Abu Dhabi,24.4667,54.3667,78.80,50,0,6.91,AE,1612435819,Beach Rotana
67,Resistencia,-27.4606,-58.9839,72.00,94,0,4.61,AR,1612435822,Niyat Urban Hotel
70,Kiryat Gat,31.6100,34.7642,71.60,46,0,6.91,IL,1612435823,OlusHome
148,Zihuatanejo,17.6333,-101.5500,73.04,78,0,4.12,MX,1612435832,La Casa Que Canta
234,Pilar,-9.5972,-35.9567,75.20,94,0,3.44,BR,1612435719,Pousada e churrascaria Manguaba
331,Puerto Escondido,15.8500,-97.0667,74.28,74,0,4.41,MX,1612435855,Aldea del Bazar Hotel and Spa
337,São João da Barra,-21.6403,-41.0511,77.00,78,0,6.91,BR,1612435856,Pousada Porto De Canoas
340,Sur,22.5667,59.5289,75.00,59,0,9.17,OM,1612435856,Sur Plaza Hotel
349,Tiarei,-17.5333,-149.3333,75.20,88,0,2.30,PF,1612435857,Le Rocher de Tahiti


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
# hotel_layer = gmaps.symbol_layer(locations)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))